In [1]:
import pandas as pd
    # import seaborn as sns
# import matplotlib.pyplot as plt
from tqdm import tqdm
from document_preprocessor import RegexTokenizer
from indexing import Indexer, IndexType, BasicInvertedIndex
from ranker import Ranker, WordCountCosineSimilarity, DirichletLM, BM25, PivotedNormalization, TF_IDF, CubeRootRanker

import relevance

import json
import ast
from datetime import datetime

In [20]:


arxiv = set()
dois = set()
new_docs = []
with open('all_papers.jsonl', 'r') as file:
    idx=0
    for line_num, line in enumerate(tqdm(file)):
        idx+=1
        document = json.loads(line.strip())
        if document.get('doi', '') == '':
            arxiv.add(document.get('arxiv_id', '')[0:10])
        else:
            doi = document.get('doi', '')
            if doi in dois:
                continue
            dois.add(doi)
            
            if 'arxiv' in doi:
                doi_2 = doi.split("/")[1][6:]
            if doi_2 in arxiv:
                continue
            
        
        new_docs.append(document)    
        

100024it [00:25, 3902.34it/s]


In [21]:
len(new_docs)

98503

In [26]:
# file_paths = ["arxiv_papers.json", "arxiv_papers_pt2.json", "OpenAlex_Text_Depth-3.json"]
output_file = "all_papers.jsonl"

# combined_data = []

# for file_path in file_paths:
#     print(f'Reading {file_path}')
#     with open(file_path, "r") as file:
#         data = json.load(file)
        
#         if file_path == "OpenAlex_Text_Depth-3.json":
#             data = [item for item in data if "text" in item and item["text"].strip()]
#             print(len(data))
        
#         combined_data.extend(data)

print('Writing the file.')
with open(output_file, "w") as jsonl_file:
    for record in new_docs:
        jsonl_file.write(json.dumps(record) + "\n")

print(f"Combined data has been written to {output_file}")


Writing the file.
Combined data has been written to all_papers.jsonl


In [27]:
54545 + 36100 #+ 9471

90645

In [2]:
from importlib import reload

import document_preprocessor
reload(document_preprocessor)
from document_preprocessor import RegexTokenizer


import indexing
reload(indexing)
from indexing import Indexer, IndexType, BasicInvertedIndex



import ranker
reload(ranker)
from ranker import Ranker, WordCountCosineSimilarity, DirichletLM, BM25, PivotedNormalization, TF_IDF, CubeRootRanker

In [3]:
docid_authors_map = {}
with open('all_papers.jsonl', 'r') as file:
    idx=0
    for line_num, line in enumerate(file):
        idx+=1
        # if idx>90643:
        document = json.loads(line.strip())
        a = document.get('authors', [])
        # print(isinstance(a, list))
        if isinstance(a, list):
            docid_authors_map[idx] = [author.lower() for author in a]
        else:
            docid_authors_map[idx] = [author.lower() for author in ast.literal_eval(a)]
        if idx%10000 == 0:
            print(f'{idx} documents done.')

10000 documents done.
20000 documents done.
30000 documents done.
40000 documents done.
50000 documents done.
60000 documents done.
70000 documents done.
80000 documents done.
90000 documents done.


In [4]:
docid_authors_map[90700] ## SAVE THIS MAPPING DICT!!!

['steffen rendle',
 'christoph freudenthaler',
 'zeno gantner',
 'lars schmidt-thieme']

In [25]:
import pickle

with open('docid_authors_map.pkl', 'wb') as file:
    pickle.dump(docid_authors_map, file)

In [5]:
all_authors = set()

for idx, auth in docid_authors_map.items():
    for a in auth:
        all_authors.add(a)

In [6]:
list(all_authors) # need to treat as Multiword Expressions!!!!!

['andy brock',
 'yifei xiao',
 'jeremy huckins',
 'gyan tatiya',
 'suho yoo',
 'janet van niekerk',
 'junhan chang',
 'finn verner jensen',
 'andy way',
 'liping si',
 'vincenzo schininà',
 'taylor henderson',
 'karn seth',
 'yuxing hu',
 'lin gu',
 'phillip b. levine',
 'sebastian tschiatschek',
 'sardana ivanova',
 'chang chen',
 'hannah metzler',
 'shari liu',
 'lars johan ersland',
 'vivienne baillie gerritsen',
 'ashley wu',
 'per sidén',
 'tanner schmidt',
 'magnus rattray',
 'ansgar philippsen',
 'abu layek',
 'elli heyes',
 'hanjun dai',
 'soohyun kim',
 'jesper b. boldt',
 'c. b. simone',
 'longjun cai',
 'drew van camp',
 'yuan xu',
 'yasir glani',
 'yadira ortiz castellano',
 'phillip maffettone',
 'mirza mohtashim alam',
 'leticia maria paz de lima',
 'jiandong zhang',
 'heather domin',
 'hanyun yin',
 'chungyi lin',
 'emma schwartzman',
 'chenfeng cao',
 'rohit mishra',
 'yulong mao',
 'qinggang zhou',
 'yinan peng',
 'marvin schiller',
 'laxmi kant',
 'dalton jones',
 'ni

In [7]:
len(all_authors)

168566

In [26]:
import pickle

with open('all_authors.pkl', 'wb') as file:
    pickle.dump(all_authors, file)

In [8]:
docid_year_map = {}

with open('all_papers.jsonl', 'r') as file:
    idx=0
    for line_num, line in enumerate(file):
        idx+=1
        if idx>0:
            document = json.loads(line.strip())
            # print(type(document.get('publish_year', '')))
            if document.get('publish_year', '') != '':
                docid_year_map[idx] = document.get('publish_year', 0)
            elif document.get('published', '') != '':
                parsed_date = datetime.strptime(document.get('published', ''), '%Y-%m-%dT%H:%M:%SZ')
                year = parsed_date.year
                docid_year_map[idx] = year
            else:
                docid_year_map[idx] = ''
            if idx%10000 == 0:
                print(f'{idx} documents done.')

10000 documents done.
20000 documents done.
30000 documents done.
40000 documents done.
50000 documents done.
60000 documents done.
70000 documents done.
80000 documents done.
90000 documents done.


In [9]:
docid_year_map[1] ## SAVE THIS MAPPING!!!

2019

In [27]:
import pickle

with open('docid_year_map.pkl', 'wb') as file:
    pickle.dump(docid_year_map, file)

In [18]:
# queries = ['Research on Reinforcement Learning techniques',
#                 'Find articles on unsupervised learning for text',
#                 'Find studies on Zero-Shot Learning',
#                 'adversarial attacks in neural networks',
#                 'federated learning',
#                 'Papers on meta-learning techniques which also talk about Reinforcement-Learning or Preference-Learning',
#                 'Find papers on Transformer Architecture published after 2010',
#                 'Find all papers published by lun ai',
#                 'Find papers by tao zeng published after 2010'
#                ]

# queries = [
#                 'Find papers on Transformer Architecture published after 2010',
#                 'Find all papers published by Ashish Vaswani',
#                 'Find papers by tao zeng or lun ai published between 2010 and 2020'
#                ]


queries = [
    "Transformers in NLP",
    "Papers on BERT",
    "Recent studies on GANs",
    "Fetch papers by Yann LeCun on Convolutional Neural Networks",
    "Show me publications by Geoffrey Hinton on Deep Learning",
    "Research on Reinforcement Learning techniques",
    "Co-authored papers by Yoshua Bengio after 2018",
    "Machine learning for healthcare applications",
    "Attention mechanisms in Computer Vision",
    "Papers on Mamba Architecture in IEEE journals",
    "Fetch recent studies on LSTM networks",
    "Show me all papers on Natural Language Processing by Christopher Manning",
    "Research on Self-Supervised Learning",
    "Articles by Jürgen Schmidhuber on Recurrent Neural Networks",
    "Recent advances in Robotics and AI",
    "Studies on Transfer Learning for computer vision tasks",
    "Show me all publications on Object Detection",
    "Fetch co-authored research by Li Fei-Fei since 2020",
    "Applications of NLP in finance",
    "Recent studies on XGBoost for classification",
    "Show me papers by Andrew Ng on Machine Learning published in 2021",
    "Retrieve papers by Ruslan Salakhutdinov on Collaborative Filtering",
    "Research on explainability in AI",
    "Papers on Fairness in Machine Learning",
    "Studies on medical image analysis using deep learning",
    "Find articles on unsupervised learning for text",
    "Papers by Tomas Mikolov on word embeddings",
    "Research on adversarial attacks in neural networks",
    "Recent studies on the Transformer model in 2023",
    "Show me papers on multi-modal learning by Pieter Abbeel",
    "Machine Learning for social good",
    "Fetch studies on reinforcement learning for robotics",
    "Recent papers on data augmentation techniques",
    "Publications on ethical AI by Timnit Gebru",
    "Explainability in neural networks",
    "Show me all co-authored work by Alex Krizhevsky on CNNs",
    "Latest research on language models in healthcare",
    'Find papers on Transformer Architecture published after 2010',
    'Find all papers published by Ashish Vaswani',
    'Find papers by tao zeng or lun ai published between 2010 and 2020'
]


In [19]:
len(queries)

40

In [14]:
with open("stopwords.txt", "r") as file:
        stopwords = [line.strip() for line in file]
    
preprocessor = RegexTokenizer()

import json

In [13]:
# print('Reading JSON file.')
# with open("arxiv_papers.json", "r") as json_file:
#     data = json.load(json_file)

# # Write data to JSONL file
# print('Converting to JSONL')
# with open("arxiv_papers.jsonl", "w") as jsonl_file:
#     for entry in data:
#         jsonl_file.write(json.dumps(entry) + "\n")


print('Creating Index')
index = Indexer.create_index(IndexType.BasicInvertedIndex, 'all_papers.jsonl', 
                             preprocessor, stopwords=stopwords, minimum_word_frequency=50)
print('Index Created')
print('Saving Index')
index.save('all_papers_index')


print('Creating Title Index')
title_index = Indexer.create_index(IndexType.BasicInvertedIndex, 'all_papers.jsonl', 
                             preprocessor, stopwords=stopwords, minimum_word_frequency=0, text_key='title')
print('Title Index Created')

print('Saving Index')
title_index.save('all_papers_title_index')

In [15]:
print('Loading Index')
index = BasicInvertedIndex()
index.load('all_papers_index')

print('Loading Title Index')
title_index = BasicInvertedIndex()
title_index.load('all_papers_title_index')

#####

Loading Index
Index loaded from all_papers_index
Loading Title Index
Index loaded from all_papers_title_index


In [20]:
BMRanker = Ranker(index, title_index, preprocessor, stopwords=stopwords, scorer=BM25(index), 
                  all_authors=list(all_authors), docid_to_authors=docid_authors_map, docid_to_year=docid_year_map)

TFIDFRanker = Ranker(index, title_index, preprocessor, stopwords=stopwords, scorer=TF_IDF(index),
                    all_authors=list(all_authors), docid_to_authors=docid_authors_map, docid_to_year=docid_year_map)

In [21]:
tfidf = []
bm25 = []

for q in queries:
    print(q)
    tfidf.append(TFIDFRanker.query(q))
    bm25.append(BMRanker.query(q))
    

Transformers in NLP


100%|██████████| 1500/1500 [00:00<00:00, 274844.09it/s]


Papers on BERT


100%|██████████| 1324/1324 [00:00<00:00, 307949.79it/s]


Recent studies on GANs


...

100%|██████████| 2816/2816 [00:00<00:00, 281358.78it/s]


Find all papers published by Ashish Vaswani


100%|██████████| 121/121 [00:00<00:00, 158498.06it/s]


Find papers by tao zeng or lun ai published between 2010 and 2020


100%|██████████| 4577/4577 [00:00<00:00, 304632.48it/s]


In [16]:
import random

In [22]:
tfidf2 = []
for i in tfidf:
    tfidf2.append(i[:20])

In [23]:
tfidf2

[[(58233, 5.658143411992613),
  (59201, 5.030026580547734),
  (51504, 4.793171721288175),
  (59105, 4.739575119150212),
  (18201, 4.719587053884814),
  (34225, 4.634700400866777),
  (57440, 4.604259605200757),
  (79684, 4.594000285338274),
  (52666, 4.580572172004343),
  (72920, 4.578745644715002),
  (56381, 4.527463473308957),
  (58138, 4.499807426771711),
  (16928, 4.437536782240654),
  (57904, 4.22874379457927),
  (59662, 4.207781449073225),
  (17207, 4.20449961843522),
  (58635, 4.202934913521098),
  (47599, 4.193351588749996),
  (41822, 4.143252313259141),
  (16985, 4.136883102339302)],
 [(52666, 5.039433035200425),
  (51586, 4.68922630111536),
  (50854, 4.5629886449296055),
  (44746, 4.525014488480336),
  (50861, 4.4743730458585915),
  (32184, 4.4399304307176015),
  (36569, 4.364270388074816),
  (50695, 4.3461780263823355),
  (33115, 4.328865636642121),
  (32888, 4.32801899319349),
  (2992, 4.320501122064888),
  (51080, 4.298696996927923),
  (51899, 4.294172938334379),
  (50855, 

In [19]:
with open('all_papers.jsonl', 'r') as file:
    idx=0
    for line_num, line in enumerate(file):
        idx+=1
        if idx==95348:
            document = json.loads(line.strip())
            print(document)
            break

{'title': 'Attention Augmented Convolutional Networks', 'doi': '10.1109/iccv.2019.00338', 'openalex_id': 'https://openalex.org/W2981413347', 'authors': ['Irwan Bello', 'Barret Zoph', 'Quoc V. Le', 'Ashish Vaswani', 'Jonathon Shlens'], 'publication_date': '2019-10-01', 'publish_year': 2019, 'keywords': ['Computer science', 'Convolutional neural network', 'Convolution (computer science)', 'Artificial intelligence', 'Feature (linguistics)', 'Neighbourhood (mathematics)', 'Pattern recognition (psychology)', 'Object detection', 'Set (abstract data type)', 'Object (grammar)', 'Baseline (sea)', 'Machine learning', 'Artificial neural network', 'Mathematics', 'Mathematical analysis', 'Philosophy', 'Linguistics', 'Programming language', 'Oceanography', 'Geology'], 'abstract': 'Convolutional networks have enjoyed much success in many computer vision applications. The convolution operation however has a significant weakness in that it only operates on a local neighbourhood, thus missing global inf

In [58]:
docid_authors_map[3488]

['jakub jakubowski',
 'natalia wojak-strzelecka',
 'rita p. ribeiro',
 'sepideh pashami',
 'szymon bobek',
 'joao gama',
 'grzegorz j nalepa']

In [14]:
tfidf3 = []
for i in tfidf2:
    tfidf3.append(random.sample(i,4))

In [15]:
tfidf3

[[(10376, 26.83205945838511),
  (5432, 26.89778253202587),
  (4731, 34.193236102163546),
  (944, 27.29377962131445)],
 [(8069, 31.88462768514792),
  (1273, 32.53567861804069),
  (3251, 33.57193979354456),
  (2397, 35.80564288002584)],
 [(19520, 24.989189315812425),
  (10931, 25.31929817140096),
  (1169, 25.816239398625164),
  (4731, 31.601511777583532)],
 [(4113, 37.27603672397212),
  (6678, 35.98090571126465),
  (13037, 35.82425533722886),
  (498, 38.84393183001212)],
 [(9092, 35.533234433401105),
  (8856, 29.85808104975241),
  (11274, 28.058224909340545),
  (9097, 27.187835555588723)],
 [(8456, 29.51662512490252),
  (254, 30.432991995082148),
  (19384, 28.032995653749612),
  (4731, 35.10656886778013)]]

In [24]:
bm252 = []
for i in bm25:
    bm252.append(i[:30])

In [25]:
bm252

[[(53070, 2.8572678768896806),
  (56381, 2.8542177337876247),
  (46577, 2.789671061684337),
  (16648, 2.7443922167334835),
  (55048, 2.701059318890985),
  (79684, 2.6399390850343525),
  (51504, 2.5071875550303138),
  (34225, 2.4408273417087374),
  (59105, 2.2319126678373156),
  (72920, 2.173646081608272),
  (59201, 2.1402022170473263),
  (36754, 2.1006932108492298),
  (63555, 1.9686947574101397),
  (1776, 1.9454869853868262),
  (17207, 1.9318846937412926),
  (58233, 1.8764594209654493),
  (58138, 1.8732431746904017),
  (57956, 1.8580589735690576),
  (53921, 1.8514766321851321),
  (47714, 1.8452128838514061),
  (31024, 1.8300648362425684),
  (51954, 1.829327177386196),
  (38752, 1.824354553726153),
  (26734, 1.8188403919665004),
  (51392, 1.8089042713453614),
  (17698, 1.8071502883674107),
  (37940, 1.8061723005580987),
  (40431, 1.8050222309376398),
  (59042, 1.7941152921152197),
  (51225, 1.7920467010253294)],
 [(51339, 2.184399018145183),
  (50700, 2.0762100863201947),
  (52494, 2.03

In [1]:
from importlib import reload


import main
reload(main)
from main import IR_Ret_Sys

In [2]:
from main import IR_Ret_Sys
IR = IR_Ret_Sys()

Loading Index
Index loaded from all_papers_index
Loading Title Index
Index loaded from all_papers_title_index
Setting up Llama Pipeline.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Initialization Complete.


In [3]:
docid_score = IR.TFIDFRanker.query('Show me papers by Ashish Vaswani on Machine Learning before 2020')
results = [
    {
    'title': IR.docid_title_map.get(i[0]),
    'link' : IR.docid_link_map.get(i[0]),
    'snippet': IR.docid_abstract_map.get(i[0])
    }
    for i in docid_score[:100]
]

100%|██████████| 96636/96636 [00:01<00:00, 89034.07it/s]


In [4]:
docid_score = IR.TFIDFRanker.query_augmented('Show me papers by Ashish Vaswani on Machine Learning before 2020')
results = [
    {
    'title': IR.docid_title_map.get(i[0]),
    'link' : IR.docid_link_map.get(i[0]),
    'snippet': IR.docid_abstract_map.get(i[0])
    }
    for i in docid_score[:100]
]

100%|██████████| 23670/23670 [00:00<00:00, 121522.84it/s]


match!
match!
match!


In [5]:
docid_score = IR.TFIDFRanker.query_llama(text_gen_pipeline=IR.text_gen_pipeline, 
                                         query='Show me papers by Ashish Vaswani on Machine Learning before 2020')
results = [
    {
    'title': IR.docid_title_map.get(i[0]),
    'link' : IR.docid_link_map.get(i[0]),
    'snippet': IR.docid_abstract_map.get(i[0])
    }
    for i in docid_score[:100]
]

LLAMA QUERY


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


query_dict: 	  {'author': ['Ashish Vaswani'], 'year': 'until 2019', 'keywords': ['machine', 'learning']}


100%|██████████| 23670/23670 [00:00<00:00, 122050.80it/s]


authors_in_query: 	 ['ashish vaswani']
relevant_years: 	 [1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]
match!
match!
match!


In [6]:
docid_score

[(279, 23.480281645675486),
 (93599, 23.474080091697076),
 (70, 23.42833686948632),
 (72, 23.35001358139103),
 (1048, 23.309322188168373),
 (530, 23.276161844372716),
 (108, 23.25085107288236),
 (9632, 23.221275037704043),
 (7624, 23.164203391071013),
 (265, 23.1493951399747),
 (373, 23.147874390200545),
 (196, 23.145867572115918),
 (251, 23.10318257268457),
 (70944, 23.10105273747341),
 (432, 23.097123759460064),
 (90216, 23.088221615338576),
 (1095, 23.084631404967244),
 (11383, 23.08114225249954),
 (598, 23.079643403870303),
 (1071, 23.077019103744046),
 (12, 23.06839845015798),
 (93721, 23.06601719354665),
 (96, 23.044412731061502),
 (1119, 23.035020944233516),
 (1150, 23.02720526812874),
 (12291, 23.02634746609553),
 (469, 23.017074226927853),
 (94111, 23.01525432610678),
 (737, 23.012321148997998),
 (69797, 23.011472985318793),
 (93065, 23.003806680873506),
 (10643, 22.98452164393269),
 (862, 22.98069719212997),
 (27, 22.97954741724126),
 (43263, 22.978955200351056),
 (1093, 22.9

In [8]:
import json
with open('all_papers.jsonl', 'r') as file:
    idx=0
    for line_num, line in enumerate(file):
        idx+=1
        if idx==279:
            document = json.loads(line.strip())
            print(document)
            break

{'arxiv_id': '1707.06742v3', 'title': 'Machine Teaching: A New Paradigm for Building Machine Learning Systems', 'authors': "['Patrice Y. Simard', 'Saleema Amershi', 'David M. Chickering', 'Alicia Edelman Pelton', 'Soroush Ghorashi', 'Christopher Meek', 'Gonzalo Ramos', 'Jina Suh', 'Johan Verwey', 'Mo Wang', 'John Wernsing']", 'summary': 'The current processes for building machine learning systems require\npractitioners with deep knowledge of machine learning. This significantly\nlimits the number of machine learning systems that can be created and has led\nto a mismatch between the demand for machine learning systems and the ability\nfor organizations to build them. We believe that in order to meet this growing\ndemand for machine learning systems we must significantly increase the number\nof individuals that can teach machines. We postulate that we can achieve this\ngoal by making the process of teaching machines easy, fast and above all,\nuniversally accessible.\n  While machine lear

In [31]:
import json
docid_title_map = {}
with open('all_papers.jsonl', 'r') as file:
    idx=0
    for line_num, line in enumerate(file):
        idx+=1
        # if idx>90643:
        document = json.loads(line.strip())
        t = document.get('title', '')
        # print(isinstance(a, list))
        # if isinstance(a, list):
        docid_title_map[idx] = t
        # else:
        #     docid_title_map[idx] = [author.lower() for author in ast.literal_eval(a)]
        if idx%10000 == 0:
            print(f'{idx} documents done.')

10000 documents done.
20000 documents done.
30000 documents done.
40000 documents done.
50000 documents done.
60000 documents done.
70000 documents done.
80000 documents done.
90000 documents done.


In [32]:
import pickle

with open('docid_title_map.pkl', 'wb') as file:
    pickle.dump(docid_title_map, file)

In [22]:
import json
docid_abstract_map = {}
with open('all_papers.jsonl', 'r') as file:
    idx=0
    for line_num, line in enumerate(file):
        idx+=1
        # if idx>90643:
        document = json.loads(line.strip())
        if document.get('abstract', '') != '':
            a = document.get('abstract', '')
        else:
            a = document.get('summary', '')
        # print(isinstance(a, list))
        # if isinstance(a, list):
        docid_abstract_map[idx] = a
        # else:
        #     docid_title_map[idx] = [author.lower() for author in ast.literal_eval(a)]
        if idx%10000 == 0:
            print(f'{idx} documents done.')

10000 documents done.
20000 documents done.
30000 documents done.
40000 documents done.
50000 documents done.
60000 documents done.
70000 documents done.
80000 documents done.
90000 documents done.


In [24]:
import pickle

with open('docid_abstract_map.pkl', 'wb') as file:
    pickle.dump(docid_abstract_map, file)

In [25]:
import json
docid_link_map = {}
with open('all_papers.jsonl', 'r') as file:
    idx=0
    for line_num, line in enumerate(file):
        idx+=1
        # if idx>90643:
        document = json.loads(line.strip())
        if document.get('global_link_openable', '') != '':
            l = document.get('global_link_openable')
        else:
            l = 'https://arxiv.org/abs/'+document.get('arxiv_id')
        # print(isinstance(a, list))
        # if isinstance(a, list):
        docid_link_map[idx] = l
        # else:
        #     docid_title_map[idx] = [author.lower() for author in ast.literal_eval(a)]
        if idx%10000 == 0:
            print(f'{idx} documents done.')

10000 documents done.
20000 documents done.
30000 documents done.
40000 documents done.
50000 documents done.
60000 documents done.
70000 documents done.
80000 documents done.
90000 documents done.


In [26]:
docid_link_map

{1: 'https://arxiv.org/abs/1909.03550v1',
 2: 'https://arxiv.org/abs/1811.04422v1',
 3: 'https://arxiv.org/abs/1707.04849v1',
 4: 'https://arxiv.org/abs/1909.09246v1',
 5: 'https://arxiv.org/abs/2301.09753v1',
 6: 'https://arxiv.org/abs/0904.3664v1',
 7: 'https://arxiv.org/abs/2012.04105v1',
 8: 'https://arxiv.org/abs/1911.06612v1',
 9: 'https://arxiv.org/abs/1909.01866v1',
 10: 'https://arxiv.org/abs/1903.08801v1',
 11: 'https://arxiv.org/abs/2108.07915v1',
 12: 'https://arxiv.org/abs/1707.09562v3',
 13: 'https://arxiv.org/abs/1907.08908v1',
 14: 'https://arxiv.org/abs/2312.03120v1',
 15: 'https://arxiv.org/abs/1507.02188v1',
 16: 'https://arxiv.org/abs/1212.2686v1',
 17: 'https://arxiv.org/abs/2001.04942v2',
 18: 'https://arxiv.org/abs/2007.01503v1',
 19: 'https://arxiv.org/abs/1906.06821v2',
 20: 'https://arxiv.org/abs/1911.00776v1',
 21: 'https://arxiv.org/abs/2011.11819v1',
 22: 'https://arxiv.org/abs/2004.00993v2',
 23: 'https://arxiv.org/abs/2009.11087v1',
 24: 'https://arxiv.or

In [27]:
import pickle

with open('docid_link_map.pkl', 'wb') as file:
    pickle.dump(docid_link_map, file)

In [ ]:
with open('all_papers.jsonl', 'r') as file:
    idx=0
    for line_num, line in enumerate(file):
        idx+=1
        if idx==95348:
            document = json.loads(line.strip())
            print(document)
            break